In [32]:
import gurobipy as gp
from gurobipy import GRB

# Quadratic (convex)

In [48]:
# Create a new model
m = gp.Model("qp")

# Create variables
x = m.addVar(ub=1.0, name="x")
y = m.addVar(ub=1.0, name="y")
z = m.addVar(ub=1.0, name="z")

# Set objective: x^2 + x*y + y^2 + y*z + z^2 + 2 x
obj = x**2 + x*y + y**2 + y*z + z**2 + 2*x
m.setObjective(obj)

# Add constraint: x + 2 y + 3 z >= 4
m.addConstr(x + 2 * y + 3 * z >= 4, "c0")

# Add constraint: x + y >= 1
m.addConstr(x + y >= 1, "c1")

<gurobi.Constr *Awaiting Model Update*>

In [49]:
m.display()

Minimize
  0.0
Subject To


In [50]:
# m.Params.NonConvex = 2 # not necessary here
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xc501370b
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Ti

In [51]:
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % obj.getValue())

x 3.255e-09
y 1
z 0.666667
Obj: 2.11111


In [47]:
m.display()

Minimize
  2.0 x + [ x ^ 2 + x * y + y ^ 2 + y * z + z ^ 2 ]
Subject To
  c0: x + 2.0 y + 3.0 z >= 4
  c1: x + y >= 1
Bounds
  0 <= x <= 1
  0 <= y <= 1
  0 <= z <= 1


In [ ]:
x.VType = GRB.INTEGER
y.VType = GRB.INTEGER
z.VType = GRB.INTEGER

m.optimize()

for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % obj.getValue())

# Bilinear (non convex)

In [40]:
# Create a new model
m = gp.Model("bilinear")

# Create variables
x = m.addVar(name="x")
y = m.addVar(name="y")
z = m.addVar(name="z")

# Set objective: maximize x
m.setObjective(1.0*x, GRB.MAXIMIZE)

# Add linear constraint: x + y + z <= 10
m.addConstr(x + y + z <= 10, "c0")

# Add bilinear inequality constraint: x * y <= 2
m.addConstr(x*y <= 2, "bilinear0")

# Add bilinear equality constraint: x * z + y * z == 1
m.addConstr(x*z + y*z == 1, "bilinear1")

<gurobi.QConstr Not Yet Added>

In [41]:
m.display()

Minimize
  0.0
Subject To


In [3]:
# First optimize() call will fail - need to set NonConvex to 2
#try:
#    m.optimize()
#except gp.GurobiError:
#    print("Optimize failed due to non-convexity")

# Solve bilinear model
m.Params.NonConvex = 2
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x83663ee2
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
  QRHS range       [1e+00, 2e+00]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 12 rows, 5 columns, 27 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 5 continuous, 0 integer (0 binary)

Root relaxation: objective 9.899370e+00, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

In [4]:
m.printAttr('x')


    Variable            x 
-------------------------
           x      9.89898 
           z     0.101021 


In [5]:
# Constrain 'x' to be integral and solve again
x.VType = GRB.INTEGER
m.optimize()
m.printAttr('x')

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x81133406
Model has 2 quadratic constraints
Variable types: 2 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
  QRHS range       [1e+00, 2e+00]

MIP start from previous solve did not produce a new incumbent solution

Presolve time: 0.00s
Presolved: 12 rows, 5 columns, 27 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 4 continuous, 1 integer (0 binary)

Root relaxation: objective 9.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objectiv

In [7]:
dir(m)

['BarIterCount',
 'BoundSVio',
 'BoundSVioIndex',
 'BoundSVioSum',
 'BoundVio',
 'BoundVioIndex',
 'BoundVioSum',
 'ComplVio',
 'ComplVioIndex',
 'ComplVioSum',
 'ConcurrentWinMethod',
 'ConcurrentWinner',
 'ConstrResidual',
 'ConstrResidualIndex',
 'ConstrResidualSum',
 'ConstrSResidual',
 'ConstrSResidualIndex',
 'ConstrSResidualSum',
 'ConstrSVio',
 'ConstrSVioIndex',
 'ConstrSVioSum',
 'ConstrVio',
 'ConstrVioIndex',
 'ConstrVioSum',
 'DNumNZs',
 'DualResidual',
 'DualResidualIndex',
 'DualResidualSum',
 'DualSResidual',
 'DualSResidualIndex',
 'DualSResidualSum',
 'DualSVio',
 'DualSVioIndex',
 'DualSVioSum',
 'DualVio',
 'DualVioIndex',
 'DualVioSum',
 'FarkasProof',
 'Fingerprint',
 'FirstSolNode',
 'HasDualNorm',
 'IISMinimal',
 'InfeasVar',
 'IntVio',
 'IntVioIndex',
 'IntVioSum',
 'IsMIP',
 'IsMultiObj',
 'IsQCP',
 'IsQP',
 'IterCount',
 'IterCount0',
 'Kappa',
 'KappaExact',
 'LicenseExpiration',
 'MIPGap',
 'MaxBound',
 'MaxCoeff',
 'MaxObjCoeff',
 'MaxQCCoeff',
 'MaxQCLCoe

In [10]:
m.NumConstrs

1

In [11]:
m.IsMIP

1

In [12]:
m.display()

Maximize
  x
Subject To
  c0: x + y + z <= 10
  bilinear0: 0.0 + [ x * y ] <= 2
  bilinear1: 0.0 + [ x * z + y * z ] >= 1
General Integers
  ['x']


In [13]:
m.write("non_linear.lp")

# PuLP (does not work for non linear problems)

In [ ]:
import pulp

In [15]:
prob=pulp.LpProblem("test",pulp.LpMaximize)

In [18]:
x=pulp.LpVariable("x",lowBound=0,upBound=5,cat=pulp.LpContinuous)

In [25]:
prob+= x
prob+= x+3<=3

In [20]:
prob

test:
MAXIMIZE
1*x + 0
SUBJECT TO
_C1: x <= 0

VARIABLES
x <= 5 Continuous

In [21]:
prob.solve()

1